# Generate and evaluate weighted voting model

### Import training and test data

In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import f1_score, mean_squared_error as mse
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from PU_Learning import *

def ignore_warnings():
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=Warning)

ignore_warnings()

In [83]:
# import training data
data = pd.read_csv('clean_data/players_17_clean_train.csv')
train_x = data.iloc[:, :-1].values
train_y = data.iloc[:, -1].values
s = data.iloc[:, -1].values
c = Counter(s)[1]/Counter(train_y)[1]

# import test data 
data = pd.read_csv('clean_data/players_17_clean_test.csv')
test_x = data.iloc[:, :-1].values
test_y = data.iloc[:, -1].values

### 1-DNFII

In [84]:
# calculate the frequency of a certain score for a certain feature
def frequency(feature, score, collection, deviation):
    count = 0
    for element in collection:
        if (element[feature] > score - deviation) and (element[feature] < score + deviation):
            count += 1
    return count

In [85]:
# returns a numpy array that holds all the elements of a that are not in b
def setdiff_nd_positivenums(a, b):
    s = np.maximum(a.max(0)+1,b.max(0)+1)
    return a[~np.isin(a.dot(s),b.dot(s))]

In [86]:
# algorithm improved 1-DNF (1-DNFII)
def dnf(train_x, train_y, deviation, parameter_lambda):
    num_rows, num_cols = train_x.shape
    
    # PF = positive feature set
    PF = np.zeros(shape=(0, 2))
    
    # RN = reliable negatives
    RN = np.zeros(shape=(0, num_cols))
    RN_y = np.array([])
    
    # U = unlabeled
    U = np.zeros(shape=(0, num_cols))
    U_y = np.array([])
    
    # P = positive
    P = np.zeros(shape=(0, num_cols))
    P_y = np.array([])
    
    # construct P, U and RN
    # at this moment: RN = U
    for i in range(0, len(train_y)-1):
        if (train_y[i] == 1):
            P = np.vstack([P, train_x[i]])
            P_y = np.append(P_y, train_y[i])
        else:
            U = np.vstack([U, train_x[i]])
            U_y = np.append(U_y, train_y[i])
            RN = np.vstack([RN, train_x[i]])
            RN_y = np.append(RN_y, train_y[i])
            
    print("1-DNFII: RN is initialized")

    # construct PF
    for i in range(0, num_cols-1):
        for j in range(0, num_rows-1):
            symbol = train_x[j][i]
            if (not [i, symbol] in PF):
                constraint_1 = frequency(i, symbol, P, deviation) / len(P)
                constraint_2 = frequency(i, symbol, U, deviation) / len(U)
                if (constraint_1 > constraint_2) and (constraint_1 > parameter_lambda):
                    new = [i, symbol]
                    PF = np.vstack([PF, new])
                
    print("1-DNFII: PF is constructed")
              
    # construct list with indices of elements to remove from RN
    # (based on PF)
    index = []        
    for i in range(0, len(RN)-1):
        for constraint in PF:
            element = RN[i]
            if element[int(constraint[0])] == constraint[1]:
                index.append(i)
              
    RN = np.delete(RN, index, 0)
    RN_y = np.delete(RN_y, index, 0)
                
    print("1-DNFII: RN was finalized")
    
    return P, P_y, U, U_y, RN, RN_y, PF

In [87]:
# deviation indicates whether two feature values are considered the same
# if deviation = 0, the two feature values need to be exactly the same
# if deviation = 1, feature value one need to be equal to feature value two - 1
#                   OR feature value one need to be equal to feature value two + 1
#                   OR the two feature values are exactly the same
# parameter_lambda = the parameter lambda used in 1-DNFII
def weighted_voting(train_x, train_y, deviation, parameter_lambda):
    P, P_y, U, U_y, RN, RN_y, PF = dnf(train_x, train_y, deviation, parameter_lambda)
    FinalWVClassifier = []
    FinalWVClassifier_weights = []

    # shuffle rows of P and then delete the first 10%
    P = np.insert(P, len(P[0]), P_y, axis=1)
    np.random.shuffle(P)
    PP = P[:round(len(P)/10),:]
    PP_y = PP[:, -1]
    PP = np.delete(PP, -1, 1)
    index = [x for x in range(0, round(len(P)/10))]
    P = np.delete(P, index, 0)
    P_y = P[:, -1]
    P = np.delete(P, -1, 1)

    PON = np.vstack([P, RN])
    PON_y = np.append(P_y, RN_y)

    RN = np.insert(RN, len(RN[0]), RN_y, axis=1)

    U = np.insert(U, len(U[0]), U_y, axis=1)
    U = setdiff_nd_positivenums(U, RN)
    U_y = U[:, -1]
    U = np.delete(U, -1, 1)

    allPrecision = 0

    last_U = 0

    while (True):
        if (len(U) == 0):
            break

        # create a new SVM
        new_clf = make_pipeline(StandardScaler(), svm.SVC(gamma='auto'))

        # round elements of PON_y to their nearest integer
        PON_y = np.rint(PON_y)

        # train the new SVM
        new_clf.fit(np.copy(PON), np.copy(PON_y))

        # predict the labels of the class U
        NEG = new_clf.predict(np.copy(U))

        # predict the labels of the class PP
        predictions = new_clf.predict(np.copy(PP))

        # calculate the precision of PP on the trained SVM
        #precision = len(predictions[predictions == 1]) / len(predictions)
        precision = f1_score(np.ones(len(predictions)), predictions)
        allPrecision += precision

        # initialize list with indexes that will be removed from U
        index = []

        # check which elements can be removed from U
        for i in range(0, len(NEG)):
            if (NEG[i] == 0):
                index.append(i)
                PON = np.vstack([PON, U[i, :]])
                PON_y = np.append(PON_y, NEG[i])

        # remove elements of U
        U = np.delete(U, index, 0)

        if (len(U) == last_U):
            break

        last_U = len(U)

        print("size U: " + str(len(U)) + " precision: " + str(precision) + " all: " + str(allPrecision))

        # add classifier to list of classifiers
        FinalWVClassifier.append(new_clf)
        FinalWVClassifier_weights.append(precision)

    for i in range(0, len(FinalWVClassifier_weights)):
        FinalWVClassifier_weights[i] = FinalWVClassifier_weights[i] / allPrecision

    print("------------------")
    print("A weighted voting method with " + str(len(FinalWVClassifier)) + " classifiers was created.")
    
    return FinalWVClassifier, FinalWVClassifier_weights

In [88]:
# predict the label of the elements of the given list with the given classifiers and weigths
# returns a list with these labels
def predict_weighted_voting_list(classifiers, weights, listt):
    result = []
    for elem in listt:
        result.append(predict_weighted_voting(classifiers, weights, elem))
    return result
    
# predict the label of x with the given classifiers and weigths
def predict_weighted_voting(classifiers, weigths, x):
    score = 0
    for i in range(0, len(classifiers)):
        score += weigths[i] * classifiers[i].predict([x])
    return round(score[0])

In [89]:
ress = []
for _ in range(10):
    name = "1-DNFII"
    classifiers, weights = weighted_voting(np.copy(train_x), np.copy(train_y), 0.4, 0.2)
    pred_y = predict_weighted_voting_list(classifiers, weights, np.copy(test_x))         
    print(name,"F1 score:", f1_score(test_y, pred_y))
    ress.append(f1_score(test_y, pred_y))
print('avg F1: ' + str(sum(ress)/len(ress)))

1-DNFII: RN is initialized
1-DNFII: PF is constructed
1-DNFII: RN was finalized
size U: 6002 precision: 0.9618768328445748 all: 0.9618768328445748
size U: 5876 precision: 0.9526627218934911 all: 1.914539554738066
size U: 5734 precision: 0.9495548961424332 all: 2.864094450880499
size U: 5396 precision: 0.9495548961424332 all: 3.813649347022932
size U: 4635 precision: 0.9464285714285715 all: 4.760077918451503
size U: 3333 precision: 0.936936936936937 all: 5.69701485538844
size U: 1986 precision: 0.897196261682243 all: 6.594211117070683
size U: 1089 precision: 0.8653846153846153 all: 7.459595732455298
size U: 656 precision: 0.8239202657807309 all: 8.283515998236028
size U: 469 precision: 0.8120805369127517 all: 9.09559653514878
size U: 393 precision: 0.7918088737201365 all: 9.887405408868917
size U: 348 precision: 0.7876712328767124 all: 10.67507664174563
size U: 328 precision: 0.7876712328767124 all: 11.462747874622343
size U: 318 precision: 0.7793103448275862 all: 12.242058219449929
siz

size U: 298 precision: 0.7535211267605634 all: 15.12054862047495
size U: 297 precision: 0.7535211267605634 all: 15.874069747235515
------------------
A weighted voting method with 19 classifiers was created.
1-DNFII F1 score: 0.7521263669501824
1-DNFII: RN is initialized
1-DNFII: PF is constructed
1-DNFII: RN was finalized
size U: 6007 precision: 0.9618768328445748 all: 0.9618768328445748
size U: 5875 precision: 0.9495548961424332 all: 1.911431728987008
size U: 5706 precision: 0.9464285714285715 all: 2.8578603004155796
size U: 5310 precision: 0.936936936936937 all: 3.7947972373525167
size U: 4231 precision: 0.9305135951661632 all: 4.72531083251868
size U: 2836 precision: 0.9240121580547112 all: 5.649322990573391
size U: 1681 precision: 0.9141104294478527 all: 6.563433420021243
size U: 952 precision: 0.8867924528301886 all: 7.450225872851432
size U: 614 precision: 0.8506493506493507 all: 8.300875223500782
size U: 455 precision: 0.8355263157894737 all: 9.136401539290256
size U: 376 preci